# MultiProcessing in Python

## 멀티 프로세싱

### 멀티프로세스란?
우선 프로세스(Processes)란 **실행 중인 프로그램**을 의미합니다. 그렇다면 멀티 프로세스란 이 프로세스를 여러개 사용한다는 의미가 됩니다. 즉, 분산처리 개념으로 이해하면 될 것 같습니다.</br>
그러면 이 방법을 왜 사용할까요? 간단한 예시로 스테이크와 스테이크 소스 두 가지를 만들어야 하는 상황일때 사용할 수 있는 가스레인지 모델이 1구인 모델과 2구인 모델의 음식 완성까지의 시간은 다르겠죠? 컴퓨터에서 하는 작업도 같은 맥락으로 이해하시면 됩니다.</br></br>

Python에서는 **multiprocessing**이라는 모듈이 내장되어 있어 이를 쉽게 구현할 수 있다고 합니다.

</br>

크롤링에 적용하기 이전에 간단한 예제로 멀티프로세싱에 대해서 알아보겠습니다.

#### 간단한 예제

In [1]:
import time

start_time = time.time()

def sleepFunction(t):
  print(f"{t} sec Sleep....")
  time.sleep(t)
  print(f'Sleep Fin {t}sec')

num_list = [1,2,3,4]


for num in num_list:
  sleepFunction(num)

print('Total Running Time : ', time.time() - start_time)

1 sec Sleep....
Sleep Fin 1sec
2 sec Sleep....
Sleep Fin 2sec
3 sec Sleep....
Sleep Fin 3sec
4 sec Sleep....
Sleep Fin 4sec
Total Running Time :  10.013209819793701


위 예시에서 보시다시피 리스트를 단순 반복문을 돌리면 앞에 작업이 끝나기 전에는 뒷 작업을 진행  할 수 없어 총 대기시간의 합인 10초가 걸렸습니다.

In [2]:
# 멀티프로세싱
from multiprocessing import Pool
import time

start_time = time.time()

def sleepFunction(t):
  print(f"{t} sec Sleep....")
  time.sleep(t)
  print(f'Sleep Fin {t}sec')

num_list = [1,2,3,4]

pool = Pool(processes=4)
pool.map(sleepFunction, num_list)

print('Total Running Time : ', time.time() - start_time)

1 sec Sleep....3 sec Sleep....2 sec Sleep....
4 sec Sleep....


Sleep Fin 1sec
Sleep Fin 2sec
Sleep Fin 3sec
Sleep Fin 4sec
Total Running Time :  4.073814630508423


멀티프로세싱으로 4초의 시간만 걸린 것을 확인 할 수 있습니다. 앞서 스테이크 예시와 같이 작업할 수 있는 공간이 늘어났기 때문에 시간이 줄어든것입니다.
</br></br>
이 멀티프로세싱에는 pool과 process 두 가지 방법이 있다고 합니다.</br>pool은 작업 전체를 던져주고 알아서 처리해! 라는 방식이고 process는 직접 넌 이거 넌 저거 이런식으로 지정해주는 것이라고합니다. 아래는 위 작업을 process로 구현한 코드입니다.

In [3]:
import multiprocessing as redfoxtistory
import time

start_time = time.time()

def sleepFunction(t):
  sec = int(t)
  print(f"{sec} sec Sleep....")
  time.sleep(sec)
  print(f'Sleep Fin {sec}sec')

p1 = redfoxtistory.Process(target = sleepFunction, args = ('1'))
p2 = redfoxtistory.Process(target = sleepFunction, args = ('2'))
p3 = redfoxtistory.Process(target = sleepFunction, args = ('3'))
p4 = redfoxtistory.Process(target = sleepFunction, args = ('4'))

#작업대에서 해야할 일을 설정해주고 실행시켜줍니다.
p1.start()
p2.start()
p3.start()
p4.start()

# join은 저희가 기존에 사용하던 str.join()과 완전 별개의 함수입니다.
# multiprocessing안에 있는 함수로 이 함수는 내가 할당한 작업대의 작업이 끝날때까지 기다려라 라는 뜻입니다.
# 아래 부분을 모두 활성화 해도 합이 아닌 가장 시간이 오래걸리는 작업대 시간만큼만 기다리는거보면 큰 작업장 개념이 따로 있나봅니다.
# 좀 더 쉽게 이해하고 싶으시면 p3만 활성화 하고 실행해보시면 p4는 작업을 완료하지 못했는데 그냥 넘어가버립니다.
# 다른 방법으로는 start와 조인을 번갈아 사용해보시면 됩니다. start -> join 하면 이 작업이 끝나야 다음 start가 실행됩니다.

p1.join() # 1초 대기
p2.join() # 2초 대기
p3.join() # 3초 대기
p4.join() # 4초 대기

print('Total Running Time : ', time.time() - start_time)

2 sec Sleep....
1 sec Sleep....
3 sec Sleep....4 sec Sleep....

Sleep Fin 1sec
Sleep Fin 2sec
Sleep Fin 3sec
Sleep Fin 4sec
Total Running Time :  4.109197378158569


#### 크롤링에 적용해보기

우선 기본적인 뷰티풀숩으로 0201일의 경제-금융의 기사들을 크롤링해보겠습니다.

In [ ]:
# 저는 useragent를 페이크로 사용했습니다. 직접 입력해주시면 이부분은 실행 안하셔도 됩니다.
!pip install fake_useragent

In [5]:
from bs4 import BeautifulSoup as bs
import requests

#useragent를 직접 입력하신다면 아래부분은 지우시고 headers부분만 직접 입력해주시면 됩니다.
from fake_useragent import UserAgent

ua = UserAgent(verify_ssl=False)
fake_ua = ua.random

headers = {
    'user-agent' : fake_ua
}

In [6]:
import time

start_time = time.time()

i=1
total_urls = []
urls = []
while True:
  test_url = f'https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=259&sid1=101&date=20230201&page={i}'
  soup = bs(requests.get(test_url, headers=headers).text, "html.parser")

  now_url =[]
  # url만 가져오기
  for row in soup.select('#main_content > div.list_body.newsflash_body > ul > li'):
    row = row.select_one('a')
    now_url.append(row['href'])
  
  if urls ==  now_url:
    break
  
  urls = now_url
  total_urls += now_url

  print(f"{i} PAGE FIN URLS CNT : ", len(urls))
  i += 1

print("-"*10)
print('END PAGE : ', i-1)
print('Total Urls : ', len(total_urls))
print('Check Running Time : ', time.time() - start_time)

1 PAGE FIN URLS CNT :  20
2 PAGE FIN URLS CNT :  20
3 PAGE FIN URLS CNT :  20
4 PAGE FIN URLS CNT :  20
5 PAGE FIN URLS CNT :  20
6 PAGE FIN URLS CNT :  20
7 PAGE FIN URLS CNT :  20
8 PAGE FIN URLS CNT :  20
9 PAGE FIN URLS CNT :  20
10 PAGE FIN URLS CNT :  20
11 PAGE FIN URLS CNT :  20
12 PAGE FIN URLS CNT :  20
13 PAGE FIN URLS CNT :  20
14 PAGE FIN URLS CNT :  20
15 PAGE FIN URLS CNT :  20
16 PAGE FIN URLS CNT :  20
17 PAGE FIN URLS CNT :  20
18 PAGE FIN URLS CNT :  20
19 PAGE FIN URLS CNT :  20
20 PAGE FIN URLS CNT :  20
21 PAGE FIN URLS CNT :  20
22 PAGE FIN URLS CNT :  20
23 PAGE FIN URLS CNT :  20
24 PAGE FIN URLS CNT :  20
25 PAGE FIN URLS CNT :  20
26 PAGE FIN URLS CNT :  20
27 PAGE FIN URLS CNT :  18
----------
END PAGE :  27
Total Urls :  538
Check Running Time :  14.20963191986084


#### 멀티프로세싱을 통한 크롤링

이번에는 멀티프로세싱으로 크롤링 해보겠습니다.

In [7]:
from multiprocessing import Pool
import time

start_time = time.time()

# 페이지 탐색을 자동으로 종료하는 방법은 떠오르지 않아 마지막 페이지 정보를 먼저 가져오도록 했습니다.
# 위 방법처럼 url들을 비교하는 방법이 있을 것 같긴한데 저도 좀 더 알아봐야할 것 같습니다. 혹시 발견하신 분은 공유해주시면 좋을 것 같아요!
def getEndPage():
  url = f'https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=259&sid1=101&date=20230201&page=999'
  soup = bs(requests.get(url, headers=headers).text, "html.parser")

  return soup.select_one('#main_content > div.paging > strong').text
  

def getUrls(page):
  test_url = f'https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=259&sid1=101&date=20230201&page={page}'
  
  res = requests.get(test_url, headers=headers)

  if res.status_code != 200:
    print(page, "page Request Error")
    return 
  
  soup = bs(res.text, "html.parser")
  now_urls =[]
  
  for row in soup.select('#main_content > div.list_body.newsflash_body > ul > li'):
      row = row.select_one('a')
      now_urls.append(row['href'])  
  
  print(f"Crawing Fin {page} URLS CNT : ", len(now_urls))
  return now_urls

pages = [i for i in range(1, int(getEndPage())+1)]

pool = Pool(processes=4)
result = pool.map(getUrls, pages)

urls = []
for url in result:
  urls += url


print("-"*10)
print('URL CNT : ', len(urls))
print('Check Running Time : ', time.time() - start_time)

Crawing Fin 3 URLS CNT :  20
Crawing Fin 7 URLS CNT :  20
Crawing Fin 1 URLS CNT :  20
Crawing Fin 5 URLS CNT :  20
Crawing Fin 4 URLS CNT :  20
Crawing Fin 8 URLS CNT :  20
Crawing Fin 6 URLS CNT :  20
Crawing Fin 2 URLS CNT :  20
Crawing Fin 9 URLS CNT :  20
Crawing Fin 11 URLS CNT :  20
Crawing Fin 13 URLS CNT :  20
Crawing Fin 15 URLS CNT :  20
Crawing Fin 10 URLS CNT :  20
Crawing Fin 12 URLS CNT :  20
Crawing Fin 17 URLS CNT :  20
Crawing Fin 19 URLS CNT :  20
Crawing Fin 14 URLS CNT :  20
Crawing Fin 18 URLS CNT :  20
Crawing Fin 16 URLS CNT :  20
Crawing Fin 20 URLS CNT :  20
Crawing Fin 21 URLS CNT :  20
Crawing Fin 23 URLS CNT :  20
Crawing Fin 27 URLS CNT :  Crawing Fin 25 URLS CNT : 18
 20
Crawing Fin 22 URLS CNT :  20
Crawing Fin 24 URLS CNT :  20
Crawing Fin 26 URLS CNT :  20
----------
URL CNT :  538
Check Running Time :  6.2536797523498535


핵심인 시간이 6초로 크게 줄어든게 눈에 보입니다. 그렇다면 작업대(processes)를 2배로 늘려보겠습니다.

In [8]:
# 위에서 작업했던 크롤링을 가져와서 그대로 사용해 보겠습니다.

from multiprocessing import Pool
import time

start_time = time.time()

def getEndPage():
  url = f'https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=259&sid1=101&date=20230201&page=999'
  soup = bs(requests.get(url, headers=headers).text, "html.parser")

  return soup.select_one('#main_content > div.paging > strong').text
  

def getUrls(page):
  test_url = f'https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=259&sid1=101&date=20230201&page={page}'
  
  res = requests.get(test_url, headers=headers)

  if res.status_code != 200:
    print(page, "page Request Error")
    return 
  
  soup = bs(res.text, "html.parser")
  now_urls =[]
  
  for row in soup.select('#main_content > div.list_body.newsflash_body > ul > li'):
      row = row.select_one('a')
      now_urls.append(row['href'])  
  
  print(f"Crawing Fin {page} URLS CNT : ", len(now_urls))
  return now_urls

pages = [i for i in range(1, int(getEndPage())+1)]

pool = Pool(processes=8)
result = pool.map(getUrls, pages)

urls = []
for url in result:
  urls += url


print("-"*10)
print('URL CNT : ', len(urls))
print('Check Running Time : ', time.time() - start_time)

Crawing Fin 2 URLS CNT : Crawing Fin 8 URLS CNT : Crawing Fin 7 URLS CNT :  Crawing Fin 1 URLS CNT :   2020 
Crawing Fin 6 URLS CNT : 20
 
20
20Crawing Fin 3 URLS CNT : 
 Crawing Fin 4 URLS CNT :  2020

Crawing Fin 5 URLS CNT : 20 
Crawing Fin 10 URLS CNT :  20
Crawing Fin 11 URLS CNT : Crawing Fin 9 URLS CNT :   2020
Crawing Fin 16 URLS CNT : Crawing Fin 12 URLS CNT :  
 2020
Crawing Fin 14 URLS CNT : 
 20
Crawing Fin 15 URLS CNT :  Crawing Fin 13 URLS CNT : 20 20

Crawing Fin 17 URLS CNT :  20
Crawing Fin 18 URLS CNT :  20
Crawing Fin 21 URLS CNT : Crawing Fin 19 URLS CNT :   20
20
Crawing Fin 24 URLS CNT :  Crawing Fin 22 URLS CNT : Crawing Fin 20 URLS CNT : 20Crawing Fin 23 URLS CNT :    2020


20
Crawing Fin 25 URLS CNT :  20
Crawing Fin 26 URLS CNT :  20
Crawing Fin 27 URLS CNT :  18
----------
URL CNT :  538
Check Running Time :  4.952283620834351


더 줄었네요! 그런데 여기서 중요한 점은 'processes'옵션의 숫자는 사용자의 컴퓨터의 성능에 따라 달라진다고 합니다.</br></br>
컴퓨터 하드웨어 중 CPU를 평가할때 8코어 16코어 이렇게 이야기하는건 광고에서 강조하는 부분이라 한 번쯤은 들어보셨을거라 생각합니다. 이게 바로 프로세스를 처리할 수 있는 작업대 수를 결정하는 거라고 합니다. 따라서 작업대를 너무 많이 설정하면 오히려 속도 저하의 원인이 된다고 합니다.

[멀티프로세스](https://superfastpython.com/multiprocessing-pool-num-workers/#What_is_a_CPU_and_What_is_a_CPU_Core)에 관련한 내용이 정리된 링크 공유로 마무리하겠습니다!</br>
